# PLEASE RUN ON GOOGLE COLAB DON'T MAKE YOUR COMPUTER CRY #

### Import Library ###

In [1]:
import cv2
import numpy as np
from keras import Model, Input
from keras.preprocessing import image
from keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics
import glob
import warnings

In [2]:
warnings.filterwarnings("ignore")

## 6.1 Data Preparation ##

In [1]:
filenames = glob.glob("./assets/face_mini/**/*.jpg")
len(filenames)

NameError: name 'glob' is not defined

In [17]:
target_size = (80,80)
all_imgs = []

for i, file in enumerate(filenames):
    img = image.load_img(file, target_size=target_size, interpolation="nearest")

    img = image.img_to_array(img)

    img = img / 255

    all_imgs.append(img)

all_imgs = np.array(all_imgs[:500])

In [18]:
all_imgs.shape

(500, 80, 80, 3)

In [19]:
x_train, x_test = train_test_split(all_imgs, random_state=32, test_size=0.3)
x_train, x_val = train_test_split(x_train, random_state=32, test_size=0.3)

In [20]:
x_train.shape, x_test.shape, x_val.shape

((245, 80, 80, 3), (150, 80, 80, 3), (105, 80, 80, 3))

In [21]:
noise_factor = 0.5
Nmean = 0
Nstd = 1

In [22]:
x_train_noisy = x_train + (noise_factor * np.random.normal(loc=Nmean, scale=Nstd, size=x_train.shape))
x_val_noisy = x_val + ( noise_factor * np.random.normal(loc=Nmean, scale=Nstd, size=x_val.shape))
x_test_noisy = x_test + ( noise_factor * np.random.normal(loc=Nmean, scale=Nstd, size=x_test.shape))

In [23]:
x_train_noisy.shape, x_test_noisy.shape, x_val_noisy.shape

((245, 80, 80, 3), (150, 80, 80, 3), (105, 80, 80, 3))

## 6.2 create_model ##

In [24]:
def create_model(optimizer="adam"):
    Input_img = Input(shape=x_train[0].shape)

    # encoding layers

    x1 = Conv2D(256, (3,3), activation="relu", padding="same")(Input_img)
    x2 = Conv2D(128, (3,3), activation="relu", padding="same")(x1)
    x2 = MaxPool2D((2,2))(x2)
    encoded = Conv2D(64, (3,3), activation="relu", padding="same")(x2)

    # decoding layers

    x3 = Conv2D(64, (3,3), activation="relu", padding="same")(encoded)
    x3 = UpSampling2D((2,2))(x3)
    x2 = Conv2D(128, (3,3), activation="relu", padding="same")(x3)
    x1 = Conv2D(256, (3,3), activation="relu", padding="same")(x2)
    decoded = Conv2D(3,(3,3), padding="same")(x1)

    # construct the autoencoder model
    autoencoder = Model(Input_img, decoded)
    autoencoder.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"])

    return autoencoder

In [25]:
model = KerasRegressor(build_fn=create_model, epochs=2, batch_size=16, verbose=0)

## 6.3 Hyperparameter Tuning ##

### Create GridSearchCV() ###

In [26]:
optimizer = ['SGD', 'RMSprop', 'Adadelta', 'Adam']
batch_size = [8, 16, 32]
epochs = [2,4,6]
param_grid = dict(batch_size=batch_size, epochs=epochs, optimizer=optimizer)

In [27]:
grid = GridSearchCV( estimator=model,verbose= 0,cv=2,n_jobs=1,param_grid =param_grid)

In [29]:
grid_result = grid.fit(x_train_noisy, x_train)

KeyboardInterrupt: 

In [ ]:
print('Best params: ',grid_result.best_params_)
print('Best score: ', grid_result.best_score_)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Create RandomizedSearchCV() ###

In [ ]:
random_search = {'optimizer': ['SGD', 'RMSprop', 'Adadelta', 'Adam'],'batch_size': list(np.linspace(8, 64, 5, dtype = int)),'epochs': list(np.linspace(1, 10, 4, dtype = int))}


In [ ]:
grid_rand = RandomizedSearchCV( estimator=model,verbose= 0,cv=2,random_state = 10,n_iter = 20,n_jobs=1,param_distributions = random_search)

NameError: name 'model' is not defined

In [ ]:
grid_rand_result = grid_rand.fit(x_train_noisy, x_train)

NameError: name 'grid_rand' is not defined

In [ ]:
print('Best params: ',grid_rand_result.best_params_)
print('Best score: ', grid_rand_result.best_score_)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

In [ ]:
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))